In [ ]:
import numpy as np
import sys
import os

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

# Absolute path to the folder that contains "quant_rotor"
sys.path.append("/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems")


#working with kietic and potential operators

# Dense (Slow)
from quant_rotor.models.dense.support_ham import write_matrix_elements, basis_m_to_p_matrix_conversion

# Sparse (Fast)
from quant_rotor.models.sparse.support_ham import build_V_in_p


# ED hamiltonian implementation.

# Dense (Slow)
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

# Sparse (Fast)
from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse


# Iterative residual calculation

# Dense (Slow)
from quant_rotor.core.dense.t_amplitudes_guess import amplitute_energy, t_1_amplitude_guess_ground_state, t_2_amplitude_guess_ground_state
from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic
from quant_rotor.core.dense.t_amplitudes_non_periodic import t_non_periodic

# Sparse (Fast)
from quant_rotor.core.dense.t_amplitudes_periodic_fast import t_periodic # for bigger systems (much harder to modify)


#Time dependant methods

# Dense (Slow)
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme

# Sparse (Fast)
from quant_rotor.core.dense.de_solve_one_thermal_dense import integration_scheme as integration_scheme_fast # for bigger systems (much harder to modify)

# Producing hamiltonian operators.

## Producing Kinetic and Potential operators.

In [2]:
state = 3
g = 0.1
tau = 0

Example of the call to produce kinetic and potential matricies.

In [3]:
# Recomended to produce small matrix which can be printed and see how it changes. For example 3 state system

Kinetic, Potential = write_matrix_elements((state-1)//2, tau) # takes input of the number of unique states see doc-string and annotation for more information

K, V = Kinetic, Potential

V_tensor = V.reshape(state, state, state, state)  # Adjust if needed

h_full = basis_m_to_p_matrix_conversion(K, state)
v_full = basis_m_to_p_matrix_conversion(V_tensor, state)

In [4]:
h_full

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j]])

In [5]:
v_full.reshape(state**2, state**2)

array([[ 0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.75+0.j, -0.25+0.j,  0.  +0.j, -0.25+0.j,  0.75+0.j],
       [ 0.  +0.j,  0.  +0.j,  0.  +0.j, -0.25+0.j,  0.  +0.j,  0.  +0.j,  0.75+0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j,  0.  +0.j,  0.75+0.j,  0.  +0.j,  0.  +0.j, -0.25+0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j, -0.25+0.j,  0.75+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.75+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [-0.25+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.75+0.j, -0.25+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [-0.25+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.75+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j]])

In [6]:
Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

# Example of how to convert to dense.

Kinetic_dense = Kinetic_sparse.toarray()
Potential_dense = Potential_sparse.toarray()

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


In [7]:
Kinetic_dense

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
nse

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  , -0.25,  0.75],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

Check the output.

In [9]:
print(np.array_equal(h_full, Kinetic_dense))
print(np.allclose(v_full.reshape(state**2, state**2), Potential_dense), 1e-30)

True
True 1e-30


Produce several matricies for tests.

In [10]:
save_dir = "example_data/kinetic_potential_matricies/ED"

for state in range(3, 15, 2):

    Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    save_path = os.path.join(save_dir, f"kin_pot_ED_state_{state}.npz")
    np.savez(save_path, K=Kinetic_dense, V=Potential_dense)

In [ ]:
save_dir = "example_data/kinetic_potential_matricies/ED"

for state in range(3, 15, 2):

    Kinetic_sparse, Potential_sparse = build_V_in_p(state, tau) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    save_path = os.path.join(save_dir, f"kin_pot_ED_state_{state}.npz")

    data = np.load(save_path)

    print(np.array_equal(Kinetic_dense, data["K"]))
    print(np.array_equal(Potential_dense, data["V"]))

True
True
True
True
True
True
True
True
True
True
True
True


## Producing hamiltonians.

In [12]:
site = 5
state = 3
g = 1

Exact Diognalisation hamiltonian.

In [13]:
H_ED, K_ED, V_ED = hamiltonian_dense(state, site, g)

eig_val_ED, eig_vec_ED = np.linalg.eigh(H_ED) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

Natural Orbitals hamiltonian.

In [14]:
H_NO, K_NO, V_NO = hamiltonian_general_dense(state, site, g)

eig_val_NO, eig_vec_NO = np.linalg.eigh(H_NO) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

[0.518991870563 0.473354148494 0.004488574708 0.002675012238 0.000416893586 0.000064196488 0.000008564889 0.00000041233  0.000000322192 0.00000000315  0.000000001362]
(11, 11)
(121, 121)
mu (121, 121)


ValueError: cannot reshape array of size 14641 into shape (9,9)

In [ ]:
np.sort(np.sort(eig_val_NO) - np.sort(eig_val_ED))

array([-0.735965074205, -0.73449777322 , -0.711856646702, -0.711177809996, -0.711155332379, -0.709785450529, -0.675014886185, -0.675011571662, -0.659616461967, -0.659616461967, -0.644559562178, -0.644553916038, -0.632508524437, -0.632508507116, -0.627855372081, -0.627855372081, -0.610428775646, -0.609877844824, -0.609877686656, -0.609603647298, -0.553318484552, -0.553311855496, -0.527148081324, -0.526956972565, -0.519519140996, -0.51951912376 , -0.519519123676, -0.519519089118, -0.518946945568, -0.51894693067 , -0.467274576685, -0.467264968848, -0.467263273161, -0.467244060239, -0.462572799989, -0.462425817479, -0.461648466619, -0.459272661472, -0.454365392036, -0.454309961983, -0.452289605083, -0.452185524894, -0.445239244223, -0.443646608422, -0.443349188756, -0.429258937402, -0.429074154255, -0.425992593261, -0.42145193964 , -0.421411374041, -0.417024725441, -0.416749548992, -0.415604945475, -0.415390217843, -0.413750541879, -0.413726030683, -0.401205848656, -0.400996971185, -0.3959